In [1]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster() 
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:40457 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 12.36 GB


In [2]:
import dask
import dask.dataframe as dd

In [3]:
cars=dd.read_csv('./../input/cars_train.csv')

In [4]:
carsTest=dd.read_csv('./../input/cars_test.csv')

In [5]:
cars.head()

37260

In [6]:
cars.compute().shape

(469992, 24)

In [7]:
cars.dtypes

Id                int64
city             object
year            float64
manufacturer     object
make             object
condition        object
cylinders        object
fuel             object
odometer        float64
title_status     object
transmission     object
drive            object
size             object
type             object
paint_color      object
lat             float64
long            float64
county_fips     float64
county_name      object
state_fips      float64
state_code       object
state_name       object
weather         float64
price             int64
dtype: object

In [8]:
cars.isnull().sum().compute()

Id                   0
city                 0
year              1713
manufacturer     37260
make             19078
condition       191021
cylinders       188076
fuel              2844
odometer        153465
title_status       708
transmission      2447
drive           180334
size            306775
type            191640
paint_color     189613
lat                  0
long                 0
county_fips      16009
county_name      16009
state_fips       16009
state_code       16009
state_name           0
weather          16182
price                0
dtype: int64

In [9]:
cars.isna().sum().compute()

Id                   0
city                 0
year              1713
manufacturer     37260
make             19078
condition       191021
cylinders       188076
fuel              2844
odometer        153465
title_status       708
transmission      2447
drive           180334
size            306775
type            191640
paint_color     189613
lat                  0
long                 0
county_fips      16009
county_name      16009
state_fips       16009
state_code       16009
state_name           0
weather          16182
price                0
dtype: int64

In [10]:
'''por lo pronto veo que hay varias columnas que se refieren a la localización, por lo que creo que voy a quedar
solo con una,stateName, que no tiene nulos y ademas tiene 52 unicos, que coincide con los 50 estados de USA,cuento
los desconocidos y corrijo un pequeño error'''
listaEstados=list(cars.state_name.unique().compute())

In [11]:
len(listaEstados)

52

In [12]:
cars[cars.state_name=='FAILED'].state_name.value_counts().compute()

FAILED    16009
Name: state_name, dtype: int64

In [13]:
"coincide con los valores de nulos en code,fips, voy a mirar las esas filas de la columnas ciudad, igual puedo sacar el estado de esas 16000"

'coincide con los valores de nulos en code,fips, voy a mirar las esas filas de la columnas ciudad, igual puedo sacar el estado de esas 16000'

In [14]:
cars[cars.state_name=='FAILED'].city.unique().compute()

0      abbotsford
1       vancouver
2         kelowna
3         nanaimo
4        winnipeg
          ...    
260      limaohio
261    providence
262    charleston
263        monroe
264        ithaca
Name: city, Length: 265, dtype: object

In [15]:
"""Hay demasiadas como para hacerlas a mano, además hay ciudades de Canada, voy a hacer el value count a ver si puedo
rescatar la mayoría a mano, resulta que las 20 con más registros cubren 13600 casos de los 16.000 y son todas
ciudades canadienses, en Canadá el volante lo tienen a la derecha, y eso es algo que influye mucho en el precio,
por lo que tenemos valor de estado, para 1000 y pico que restan creo que merece la pena tambien categorizarlas
como canada, antes que desconocido, porque me da que vamos a acertar más que fallar"""
cars[cars.state_name=='FAILED'].city.value_counts().head(10).sum()


13696

In [16]:
cars[cars.state_name=='FAILED'].city.value_counts().head(20).index,

(Index(['vancouver', 'abbotsford', 'whistler', 'victoria', 'nanaimo', 'kelowna',
        'comoxvalley', 'kamloops', 'toronto', 'sunshine', 'princegeorge',
        'montreal', 'kootenays', 'calgary', 'edmonton', 'cariboo', 'ftmcmurray',
        'ottawa', 'skeena', 'saskatoon'],
       dtype='object'),)

In [17]:
#Todas son ciudades de canada
from preparingDf import cleanNameStateName,dropLocationColumns
cars.state_name=cars.state_name.apply(cleanNameStateName)
cars= dropLocationColumns(cars)

/home/curro/Repos/clean-p/env/lib/python3.6/site-packages/dask/dataframe/core.py:2957: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('state_name', 'object'))

  warnings.warn(meta_warning(meta))


In [18]:
cars.head()

,Id,year,manufacturer,make,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state_name,weather,price
0,559327,2006.0,NaN,Scion tc,excellent,4 cylinders,gas,190000.0,clean,automatic,NaN,NaN,NaN,blue,Maryland,56.0,3200
1,1429566,2018.0,dodge,charger sxt,NaN,NaN,gas,NaN,clean,automatic,NaN,NaN,NaN,NaN,Illinois,48.0,30620
2,931606,1997.0,ford,f 250 2 wheel dr pickup,fair,NaN,gas,NaN,clean,automatic,NaN,NaN,pickup,white,Michigan,45.0,1800
3,1265412,2003.0,ram,NaN,NaN,8 cylinders,gas,NaN,clean,automatic,NaN,NaN,NaN,NaN,Texas,67.0,4500
4,1133731,2000.0,dodge,1500,NaN,NaN,gas,NaN,clean,automatic,NaN,NaN,NaN,NaN,Florida,65.0,1400


In [19]:
len(cars.state_name.unique())#50 estados de USA + un valor para canadá

51

In [20]:
#vamos con los años,
cars.year.value_counts().compute()

2007.0    29252
2008.0    28085
2006.0    27597
2015.0    25785
2005.0    25316
          ...  
1902.0        2
1913.0        1
1912.0        1
1909.0        1
1907.0        1
Name: year, Length: 115, dtype: int64

In [21]:
from preparingDf import fillnaYear,cleanYear
cars = fillnaYear(cars)

In [22]:
"""115 años me cuadra, esta columna me cuadra, los perdidos lo voy a inicializar a 0.0, y despues voy a hacer un
cast a string porque quiero tener esta columna como object para cuando llegue la fase de preparación"""

cars.year=cars.year.apply(cleanYear,meta=('year', 'object'))

In [23]:
cars.dtypes

Id                int64
year             object
manufacturer     object
make             object
condition        object
cylinders        object
fuel             object
odometer        float64
title_status     object
transmission     object
drive            object
size             object
type             object
paint_color      object
state_name       object
weather         float64
price             int64
dtype: object

In [24]:
from preparingDf import cleanManufacturer1
cars.manufacturer=cars.manufacturer.apply(cleanManufacturer1)
listaMarcas=sorted(list(cars.manufacturer.unique().compute()))

/home/curro/Repos/clean-p/env/lib/python3.6/site-packages/dask/dataframe/core.py:2957: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('manufacturer', 'object'))

  warnings.warn(meta_warning(meta))


In [30]:
listaMarcas.remove('nan')

ValueError: list.remove(x): x not in list

In [40]:
cars=cars.apply(lambda row:cleanManufacturer2(row,listaMarcas),axis=1)

In [42]:
cars.manufacturer.value_counts()

ford            80180
chevrolet       71465
unknown         35906
toyota          31788
honda           29554
nissan          21961
dodge           20828
jeep            20386
gmc             19309
ram             15739
bmw             11908
volkswagen      10351
hyundai          9098
chrysler         8678
mercedes         8580
subaru           8110
cadillac         6829
kia              6807
mazda            6545
buick            6285
pontiac          6094
lexus            5539
audi             4104
lincoln          3561
infiniti         3259
volvo            2996
mitsubishi       2950
mercury          2800
saturn           2608
mini             1759
rover            1520
jaguar           1194
fiat              560
harley            207
datsun            164
infinity          152
alfa-romeo         63
ferrari            62
landrover          34
porche             28
aston-martin       27
morgan              4
Name: manufacturer, dtype: int64

In [3]:
from preparingDf import cleanAllDf
cars=cleanAllDf()

/home/curro/Repos/clean-p/env/lib/python3.6/site-packages/dask/dataframe/core.py:2957: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('state_name', 'object'))

  warnings.warn(meta_warning(meta))
/home/curro/Repos/clean-p/env/lib/python3.6/site-packages/dask/dataframe/core.py:2957: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(

In [4]:
cars.head()

,Id,year,manufacturer,make,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state_name,weather,price
0,559327,2006,unknown,Scion tc,excellent,4 cylinders,gas,190000.0,clean,automatic,NaN,NaN,NaN,blue,Maryland,56.0,3200
1,1429566,2018,dodge,charger sxt,NaN,NaN,gas,NaN,clean,automatic,NaN,NaN,NaN,NaN,Illinois,48.0,30620
2,931606,1997,ford,f 250 2 wheel dr pickup,fair,NaN,gas,NaN,clean,automatic,NaN,NaN,pickup,white,Michigan,45.0,1800
3,1265412,2003,ram,NaN,NaN,8 cylinders,gas,NaN,clean,automatic,NaN,NaN,NaN,NaN,Texas,67.0,4500
4,1133731,2000,dodge,1500,NaN,NaN,gas,NaN,clean,automatic,NaN,NaN,NaN,NaN,Florida,65.0,1400


In [8]:
cars=cars.drop(columns='make')#arriesgado, pero está muy sucia

In [9]:
cars.isna().sum().compute()

Id                   0
year                 0
manufacturer         0
condition       191021
cylinders       188076
fuel              2844
odometer        153465
title_status       708
transmission      2447
drive           180334
size            306775
type            191640
paint_color     189613
state_name           0
weather          16182
price                0
dtype: int64

In [11]:
carsTest=dd.read_csv('./../input/cars_test.csv')
carsTest.isna().sum().compute()

Id                   0
city                 0
year               911
manufacturer     20048
make             10112
condition       102695
cylinders       101146
fuel              1583
odometer         83110
title_status       373
transmission      1350
drive            96866
size            164839
type            103023
paint_color     101787
lat                  0
long                 0
county_fips       8632
county_name       8632
state_fips        8632
state_code        8632
state_name           0
weather           8716
dtype: int64

In [16]:
cars[cars.cylinders.isna()&cars.condition.isna()].compute().shape

(147479, 16)

In [32]:
from preparingDf import cleanAllDf
cars=cleanAllDf()

/home/curro/Repos/clean-p/env/lib/python3.6/site-packages/dask/dataframe/core.py:2957: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('state_name', 'object'))

  warnings.warn(meta_warning(meta))
/home/curro/Repos/clean-p/env/lib/python3.6/site-packages/dask/dataframe/core.py:2957: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(

In [33]:
cars.head()

,Id,year,manufacturer,make,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state_name,weather,price
0,559327,2006,unknown,Scion tc,excellent,4 cylinders,gas,190000.0,clean,automatic,NaN,NaN,NaN,blue,Maryland,56.0,3200
1,1429566,2018,dodge,charger sxt,NaN,NaN,gas,NaN,clean,automatic,NaN,NaN,NaN,NaN,Illinois,48.0,30620
2,931606,1997,ford,f 250 2 wheel dr pickup,fair,NaN,gas,NaN,clean,automatic,NaN,NaN,pickup,white,Michigan,45.0,1800
3,1265412,2003,ram,NaN,NaN,8 cylinders,gas,NaN,clean,automatic,NaN,NaN,NaN,NaN,Texas,67.0,4500
4,1133731,2000,dodge,1500,NaN,NaN,gas,NaN,clean,automatic,NaN,NaN,NaN,NaN,Florida,65.0,1400


In [34]:
cars.condition=cars.condition.fillna('unknown')

In [36]:
cars.condition.value_counts().compute()

unknown      191021
excellent    120909
good         103297
like new      30469
fair          20214
new            2437
salvage        1645
Name: condition, dtype: int64

In [37]:
def cleanCondition(condition):
    if condition in ['like new','fair']:
        return 'excellent'
    elif condition in ['salvage']:
        return 'unknown'
    return condition

In [38]:
cars.condition=cars.condition.apply(cleanCondition)

/home/curro/Repos/clean-p/env/lib/python3.6/site-packages/dask/dataframe/core.py:2957: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('condition', 'object'))

  warnings.warn(meta_warning(meta))


In [40]:
cars.condition.value_counts().unique().compute()

0    192666
1    171592
2    103297
3      2437
Name: condition, dtype: int64

In [41]:
cars= cars.compute()

In [43]:
cars.to_csv('../output/partialClean.csv')